In [1]:
# from __future__ import absolute_import, division, print_function, unicode_literals

# from tensorflow.keras.preprocessing import sequence
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Embedding, Dropout, BatchNormalization
# from tensorflow.keras.datasets import imdb
# from keras.datasets import boston_housing
import numpy as np
import tensorflow as tf
from tensorflow import keras

import os
import sys
sys.path.append('..')  
from GradPri_utils.utils import *
from tensorflow.keras.preprocessing import image

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [54]:
DATA_PATH_train = 'ants_bees/train'
file_name_train_ants = getfile_name(os.path.join(DATA_PATH_train, 'ants'))
file_name_train_bees = getfile_name(os.path.join(DATA_PATH_train, 'bees'))
input_size = 224 #224 for Vgg19 and ResNet101V2, 150 for Xception, 299 for InceptionV3

trainX = []
trainY = []
for i in range(len(file_name_train_ants)):
    img_path = os.path.join(DATA_PATH_train, 'ants', file_name_train_ants[i])
    img = image.load_img(img_path, target_size=(input_size, input_size))
    x_tmp = image.img_to_array(img)
    trainX.append(x_tmp)
    trainY.append(0)

for i in range(len(file_name_train_bees)):
    img_path = os.path.join(DATA_PATH_train, 'bees', file_name_train_bees[i])
    img = image.load_img(img_path, target_size=(input_size, input_size))
    x_tmp = image.img_to_array(img)
    trainX.append(x_tmp)
    trainY.append(1)

In [55]:
DATA_PATH_test = 'ants_bees/test'
file_name_test_ants = getfile_name(os.path.join(DATA_PATH_test, 'ants'))
file_name_test_bees = getfile_name(os.path.join(DATA_PATH_test, 'bees'))

testX = []
testY = []
for i in range(len(file_name_test_ants)):
    img_path = os.path.join(DATA_PATH_test, 'ants', file_name_test_ants[i])
    img = image.load_img(img_path, target_size=(input_size, input_size))
    x_tmp = image.img_to_array(img)
    testX.append(x_tmp)
    testY.append(0)

for i in range(len(file_name_test_bees)):
    img_path = os.path.join(DATA_PATH_test, 'bees', file_name_test_bees[i])
    img = image.load_img(img_path, target_size=(input_size, input_size))
    x_tmp = image.img_to_array(img)
    testX.append(x_tmp)
    testY.append(1)

In [56]:
trainX = np.array(trainX)
trainY = np.array(trainY)
testX = np.array(testX)
testY = np.array(testY)

In [57]:
print(np.shape(trainX), np.shape(trainY), np.shape(testX), np.shape(testY))

(244, 224, 224, 3) (244,) (153, 224, 224, 3) (153,)


In [67]:
# # First, instantiate a base model with pre-trained weights.
# base_model = keras.applications.Xception(
#     weights='imagenet',  # Load weights pre-trained on ImageNet.
#     input_shape=(input_size, input_size, 3),
#     include_top=False)  # Do not include the ImageNet classifier at the top.

# base_model = keras.applications.InceptionV3(
#     weights='imagenet',  # Load weights pre-trained on ImageNet.
#     input_shape=(input_size, input_size, 3),
#     include_top=False)  # Do not include the ImageNet classifier at the top.

# base_model = keras.applications.ResNet101V2(
#     weights='imagenet',  # Load weights pre-trained on ImageNet.
#     input_shape=(input_size, input_size, 3),
#     include_top=False)  # Do not include the ImageNet classifier at the top.

base_model = keras.applications.VGG19(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(input_size, input_size, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

In [68]:
# Then, freeze the base model.
base_model.trainable = False

In [69]:
# Create a new model on top.
inputs = keras.Input(shape=(input_size, input_size, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors

x = keras.layers.GlobalAveragePooling2D()(x)

# x = keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(x)
# x = keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(x)
# x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
# x = keras.layers.Flatten()(x)
# x = keras.layers.Dense(1024, activation='relu')(x)
# x = keras.layers.Dense(128, activation='relu')(x)

# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
global_average_pooling2d_11  (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 513       
Total params: 20,024,897
Trainable params: 513
Non-trainable params: 20,024,384
_________________________________________________________________


In [70]:
# Train the model on new data.
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

model.fit(x=trainX, y=trainY, batch_size=16, epochs=20, shuffle=True,
         validation_data=(testX, testY))

Epoch 1/20
16/16 [==============================] - 2s 78ms/step - loss: 1.8606 - binary_accuracy: 0.5000 - val_loss: 1.3810 - val_binary_accuracy: 0.6471
Epoch 2/20
16/16 [==============================] - 0s 22ms/step - loss: 1.0261 - binary_accuracy: 0.6844 - val_loss: 0.9686 - val_binary_accuracy: 0.6797
Epoch 3/20
16/16 [==============================] - 0s 23ms/step - loss: 0.6661 - binary_accuracy: 0.7541 - val_loss: 0.7809 - val_binary_accuracy: 0.7778
Epoch 4/20
16/16 [==============================] - 0s 24ms/step - loss: 0.5025 - binary_accuracy: 0.8361 - val_loss: 0.7237 - val_binary_accuracy: 0.7908
Epoch 5/20
16/16 [==============================] - 1s 43ms/step - loss: 0.4260 - binary_accuracy: 0.8238 - val_loss: 0.6818 - val_binary_accuracy: 0.8366
Epoch 6/20
16/16 [==============================] - 0s 24ms/step - loss: 0.3347 - binary_accuracy: 0.8770 - val_loss: 0.6833 - val_binary_accuracy: 0.8235
Epoch 7/20
16/16 [==============================] - 0s 26ms/step - los

In [71]:
model.evaluate(x=trainX, y=trainY)

8/8 [==============================] - 1s 22ms/step - loss: 0.0544 - binary_accuracy: 0.9918


[0.05435030534863472, 0.9918032884597778]

In [72]:
model.evaluate(x=testX, y=testY)

5/5 [==============================] - 0s 20ms/step - loss: 0.6349 - binary_accuracy: 0.8235


[0.6349330544471741, 0.8235294222831726]